### Create workgroup for Redshift Serverless

#### Set up dependencies

In [ ]:
import boto3
import configparser
import json

from botocore.exceptions import ClientError

In [ ]:
aws_config = configparser.ConfigParser()
aws_config.read_file(open("access_key.config"))

AWS_ACCESS_KEY = aws_config.get("AWS", "KEY")
AWS_SECRET_ACCESS_KEY = aws_config.get("AWS", "SECRET")
REGION_NAME = "ap-northeast-1"

In [ ]:
warehouse_config = configparser.ConfigParser()
warehouse_config.read_file(open("warehouse.config"))

SECURITY_GROUP_NAME = warehouse_config.get("security-group", "SECURITY_GROUP_NAME")
SECURITY_GROUP_DESCRIPTION = warehouse_config.get("security-group", "SECURITY_GROUP_DESCRIPTION")

ROLE_NAME = warehouse_config.get("role", "ROLE_NAME")

WAREHOUSE_NAMESPACE = warehouse_config.get("warehouse", "WAREHOUSE_NAMESPACE")
WAREHOUSE_USERNAME= warehouse_config.get("warehouse", "WAREHOUSE_USERNAME")
WAREHOUSE_DB = warehouse_config.get("warehouse", "WAREHOUSE_DB")
WAREHOUSE_PASSWORD = warehouse_config.get("warehouse", "WAREHOUSE_PASSWORD")
WAREHOUSE_PORT = warehouse_config.get("warehouse", "WAREHOUSE_PORT")
WAREHOUSE_WORKGROUP = warehouse_config.get("warehouse", "WAREHOUSE_WORKGROUP")
WAREHOUSE_BASE_CAPACITY = warehouse_config.get("warehouse", "WAREHOUSE_BASE_CAPACITY")
WAREHOUSE_MAX_CAPACITY = warehouse_config.get("warehouse", "WAREHOUSE_MAX_CAPACITY")
WAREHOUSE_ENHANCED_VPC_ROUTING = warehouse_config.get("warehouse", "WAREHOUSE_ENHANCED_VPC_ROUTING")
WAREHOUSE_PUBLIC_ACCESSIBLE = warehouse_config.get("warehouse", "WAREHOUSE_PUBLIC_ACCESSIBLE")

In [ ]:
s3 = boto3.resource(
    "s3",
    region_name=REGION_NAME,
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)
iam = boto3.client(
    "iam",
    region_name=REGION_NAME,
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)
iam_resource = boto3.resource(
    "iam",
    region_name=REGION_NAME,
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)
redshift = boto3.client(
    "redshift-serverless",
    region_name=REGION_NAME,
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)
ec2 = boto3.resource(
    "ec2",
    region_name=REGION_NAME,
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)
ec2_client = boto3.client(
    'ec2',
    region_name=REGION_NAME,
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

#### Create security group

In [ ]:
vpcs = ec2_client.describe_vpcs()
vpc_id = vpcs.get('Vpcs', [{}])[0].get('VpcId', '')
try:
    response = ec2.create_security_group(
        GroupName=SECURITY_GROUP_NAME,
        Description=SECURITY_GROUP_DESCRIPTION,
        VpcId=vpc_id
    )
    groups = ec2_client.describe_security_groups(
        Filters=[
            dict(Name='group-name', Values=[SECURITY_GROUP_NAME])
        ]
    )
    SECURITY_GROUP_ID = groups['SecurityGroups'][0]['GroupId']
    data = ec2_client.authorize_security_group_ingress(
        GroupId=SECURITY_GROUP_ID,
        IpPermissions=[
            {'IpProtocol': 'TCP',
             'FromPort': int(WAREHOUSE_PORT),
             'ToPort': int(WAREHOUSE_PORT),
             'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
        ])
except ClientError as e:
    print(e)

#### Create IAM Roles

In [ ]:
try:
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "redshift.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    s3_access_policy_arn = "arn:aws:iam::aws:policy/AmazonS3FullAccess"
    role = iam.create_role(
        RoleName=ROLE_NAME, AssumeRolePolicyDocument=json.dumps(trust_policy)
    )
    iam_resource.Policy(s3_access_policy_arn).attach_role(RoleName=ROLE_NAME)
except ClientError as e:
    print(e)

#### Create Namespace

In [ ]:
try:
    role_arn = iam.get_role(RoleName=ROLE_NAME)["Role"]["Arn"]
    create_namespace = redshift.create_namespace(
        adminUserPassword=WAREHOUSE_PASSWORD,
        adminUsername=WAREHOUSE_USERNAME,
        dbName=WAREHOUSE_DB,
        defaultIamRoleArn=role_arn,
        iamRoles=[role_arn],
        logExports=['useractivitylog'],
        manageAdminPassword=False,
        namespaceName=WAREHOUSE_NAMESPACE,
    )
except Exception as e:
    print(e)

#### Create Workgroup

In [ ]:
try:
    subnets = ec2_client.describe_subnets()
    # Extract subnet IDs from the response
    subnet_ids = [subnet['SubnetId'] for subnet in subnets['Subnets']]
    
    response = redshift.create_workgroup(
        baseCapacity=int(WAREHOUSE_BASE_CAPACITY),
        enhancedVpcRouting=bool(WAREHOUSE_ENHANCED_VPC_ROUTING),
        maxCapacity=int(WAREHOUSE_MAX_CAPACITY),
        namespaceName=WAREHOUSE_NAMESPACE,
        port=int(WAREHOUSE_PORT),
        publiclyAccessible=bool(WAREHOUSE_PUBLIC_ACCESSIBLE),
        securityGroupIds=[SECURITY_GROUP_ID],
        subnetIds=subnet_ids,
        workgroupName=WAREHOUSE_WORKGROUP
    )
except Exception as e:
    print(e)

### Connect and create table in Redshift Serverless

In [ ]:
import psycopg2

In [ ]:
workgroup = redshift.get_workgroup(workgroupName=WAREHOUSE_WORKGROUP)["workgroup"]
WAREHOUSE_ENDPOINT = workgroup["endpoint"]["address"]

In [ ]:
try:
    conn = psycopg2.connect(
        host=WAREHOUSE_ENDPOINT,
        dbname=WAREHOUSE_DB,
        user=WAREHOUSE_USERNAME,
        password=WAREHOUSE_PASSWORD,
        port=int(WAREHOUSE_PORT)
    )
except Exception as e:
    print(e)

conn.set_session(autocommit=True)

In [ ]:
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print(e)

In [ ]:
try:
    cur.execute("""
        create table users(
            id integer not null distkey sortkey,
            firstname varchar(100),
            lastname varchar(100),
            email varchar(100),
            username varchar(100),
            phone varchar(100)
        );
    """)
except psycopg2.Error as e:
    print(e)

In [ ]:
BUCKET_NAME = "chase--test"
OBJECT_NAME = "user_pipeline.csv"

In [ ]:
try:
    cur.execute("""
        copy users from 's3://{BUCKET_NAME}/{OBJECT_NAME}'
        credentials 'aws_iam_role={ROLE_ARN}'
        delimiter ','
        region '{REGION_NAME}'
    """.format(
        BUCKET_NAME=BUCKET_NAME,
        OBJECT_NAME=OBJECT_NAME,
        ROLE_ARN=role_arn,
        REGION_NAME=REGION_NAME
    ))
except psycopg2.Error as e:
    print(e)

In [ ]:
try:
    cur.execute("""
        select * from users;
    """)
except psycopg2.Error as e:
    print(e)    

In [ ]:
row = cur.fetchone()
print(row)

<p><b>Go to Redshift to check the results</b></p>

#### Remove all services

In [ ]:
try:
    redshift.delete_workgroup(
        workgroupName=WAREHOUSE_WORKGROUP
    )
except ClientError as e:
    print(e)

In [ ]:
try:
    redshift.delete_namespace(
        namespaceName=WAREHOUSE_NAMESPACE
    )
except ClientError as e:
    print(e)

In [ ]:
try:
    iam_resource.Policy(s3_access_policy_arn).detach_role(RoleName=ROLE_NAME)
    iam_resource.Role(ROLE_NAME).delete()
except ClientError as e:
    print(e)

In [ ]:
try:
    ec2_client.delete_security_group(GroupId=SECURITY_GROUP_ID)
except ClientError as e:
    print(e)